In [1]:
import csv
import os
from pprint import pprint
from collections import defaultdict
import pandas as pd
import json
import yaml
from omid_openalex.utils import read_csv_tables
from pprint import pprint
import re 
from tqdm import tqdm
import psycopg2 as pg
import logging

In [2]:
# load multi mapped works into a dataframe

df = pd.read_csv('../mm_latest/multi_mapped_omids.csv')
print(len(df))

173513


In [3]:
# get df rows with S in openalex_id
# df[~(df['type'].isin(['book', 'journal article', 'journal']))]
df[df['type'].isin(['journal article'])]
df[df.apply(lambda x: x['type'] == 'journal article' and len(x['openalex_id'].split()) > 6, axis=1)]


,omid,openalex_id,type
274,omid:br/062103847820,W3130231903 W2768614374 W4246116802 W274183670...,journal article
298,omid:br/062103847576,W4245034526 W4281560519 W4247988456 W424078887...,journal article
908,omid:br/0603911303,W3130231903 W2768614374 W4246116802 W274183670...,journal article
912,omid:br/0603911552,W2063068873 W2071233178 W2036143407 W207647548...,journal article
988,omid:br/06304276376,W4285365171 W4285449806 W4214771627 W428055392...,journal article
...,...,...,...
171068,omid:br/06901893692,W4240898236 W4237384995 W2987261353 W423160785...,journal article
171145,omid:br/0690328261,W2436631477 W2400069297 W2471123172 W241805949...,journal article
171694,omid:br/062403872516,W4210489458 W4256466849 W4239045377 W425157145...,journal article
172603,omid:br/062403814971,W2116316296 W4243035123 W4241622834 W425597291...,journal article


In [13]:
# print rows with more than 100 openalex ids
df[df.apply(lambda x: len(x['openalex_id'].split()) > 100, axis=1)]
    

,omid,openalex_id,type
32178,omid:br/062203870150,W4242283090 W4249901303 W4235738792 W424136144...,journal article
32573,omid:br/06204240853,W4250798890 W4251225402 W4252304688 W423364421...,NaN
41773,omid:br/061103879754,W4245489163 W4244176929 W4206190721 W424556542...,journal article
48559,omid:br/062203824897,W4255349642 W4241759242 W4247867634 W423246104...,journal article
54273,omid:br/06204301305,W4241252030 W4241417271 W4237657838 W424592378...,dataset
59005,omid:br/062203816111,W4253849139 W4241372305 W4232032533 W424948290...,journal article
72967,omid:br/062203853413,W4241249508 W4235012619 W4250727406 W425267605...,journal article
83889,omid:br/06403991825,W2120959236 W1970692848 W4234280043 W199837802...,journal article
105255,omid:br/06604256663,W4252444989 W4239333056 W2432954410 W425640327...,journal article
105945,omid:br/062403894698,W4225531517 W4237431566 W4241824966 W430187245...,journal article


In [10]:
test = 'https://doi.org/10.21203/rs.2.488/v2'
test2 = 'https://doi.org/10.21203/rs.2.488.v14.1'
test3 = 'https://doi.org/10.21203/rs.2.488v14'

version_pattern = re.compile(r'(?:[\.\/]v\d{1,2}[\./])|(?:[\.\/]v\d{1,2}$)|(?:\/\d{1,2}$)|(?:[^a-zA-Z]v\d{1,2}$)')

print(re.findall(version_pattern, test))

['/v2']


In [47]:
import pandas as pd

data = {
    "works": {
        "proceedings article": {"D": 477, "ABC": 10, "non classified": 1118, "A": 108, "possible_preprints": 16},
        "journal article": {"non classified": 86323, "A": 10196, "ABC": 8721, "D": 38179, "possible_preprints": 1031, "EFG": 805},
        "": {"A": 1753, "D": 607, "non classified": 2112, "ABC": 502, "possible_preprints": 265, "EFG": 29},
        "book chapter": {"non classified": 3114, "EFG": 36, "D": 341, "A": 21, "ABC": 8, "possible_preprints": 4},
        "book": {"non classified": 9092, "A": 31, "D": 27, "ABC": 1, "EFG": 4},
        "proceedings": {"non classified": 738, "A": 13, "D": 8, "ABC": 24, "possible_preprints": 1},
        "report": {"non classified": 322, "ABC": 1, "D": 13, "A": 1},
        "web content": {"ABC": 146, "A": 335, "non classified": 61, "possible_preprints": 58, "D": 2},
        "reference entry": {"D": 99, "non classified": 79, "ABC": 7, "EFG": 13, "possible_preprints": 1},
        "reference book": {"non classified": 76},
        "dataset": {"non classified": 48, "A": 37, "D": 1},
        "dissertation": {"non classified": 18, "possible_preprints": 1, "A": 1},
        "standard": {"non classified": 7},
        "series": {"non classified": 7},
        "journal": {"D": 4},
        "book section": {"non classified": 1},
    },
    "sources": {
        "journal": {"A": 4057, "non classified": 2345},
        "book series": {"non classified": 38, "A": 17},
        "series": {"A": 2},
    },
}

# Create a DataFrame for "works"
works_data = {}
for work_type, work_values in data["works"].items():
    works_data[work_type] = work_values

works_df = pd.DataFrame(works_data).T.fillna(0).astype(int)



# Create a DataFrame for "sources"
sources_data = {}
for source_type, source_values in data["sources"].items():
    sources_data[source_type] = source_values

sources_df = pd.DataFrame(sources_data).T.fillna(0).astype(int)

In [48]:
# Display the resulting DataFrames
print("Works DataFrame:")
works_df



Works DataFrame:


,D,ABC,non classified,A,possible_preprints,EFG
proceedings article,477,10,1118,108,16,0
journal article,38179,8721,86323,10196,1031,805
,607,502,2112,1753,265,29
book chapter,341,8,3114,21,4,36
book,27,1,9092,31,0,4
proceedings,8,24,738,13,1,0
report,13,1,322,1,0,0
web content,2,146,61,335,58,0
reference entry,99,7,79,0,1,13
reference book,0,0,76,0,0,0


In [41]:
print("\nSources DataFrame:")
sources_df


Sources DataFrame:


,A,non classified
journal,4057,2345
book series,17,38
series,2,0
